In [1]:
from src.config import BLD
from src.config import SRC
import pandas as pd
import numpy as np
from pathlib import Path
""" from src.utils import remove_brackets_in_categorical_values
from src.data_management.cleaning_helpers.renaming_replacing import set_types_file
from src.data_management.cleaning_helpers.data_checks import general_data_checks """

' from src.utils import remove_brackets_in_categorical_values\nfrom src.data_management.cleaning_helpers.renaming_replacing import set_types_file\nfrom src.data_management.cleaning_helpers.data_checks import general_data_checks '

In [2]:
df_h=pd.read_stata("src/original_data/HHENDDAT_cf_W11.dta",convert_categoricals=False)

In [3]:
df_p=pd.read_stata("src/original_data/PENDDAT_cf_W11.dta",convert_categoricals=False)

In [4]:
df_h_c = pd.read_pickle(BLD/"HHENDDAT_clean.pickle")


In [11]:
df_p_c = pd.read_pickle(BLD/"PENDDAT_clean.pickle")


In [122]:
df_h_c["HLS0900b"] == 1

hh_id        wave
10000019.0   1.0     False
             3.0     False
10000020.0   1.0     False
             2.0     False
             3.0     False
                     ...  
110002040.0  11.0    False
110002041.0  11.0    False
110002042.0  11.0     True
110002045.0  11.0    False
110002057.0  11.0    False
Name: HLS0900b, Length: 17874, dtype: bool

In [6]:
dummy_b=pd.get_dummies(df_h_c['HLS0900b'],prefix="HLS0900b",dummy_na=True)
dummy_b.loc[dummy_b.HLS0900b_nan==1,["HLS0900b_1.0","HLS0900b_2.0"]]=np.nan
dummy_b



HLS0900b_1.0  HLS0900b_2.0  HLS0900b_nan
hh_id       wave                                          
10000019.0  1.0            NaN           NaN             1
            3.0            NaN           NaN             1
10000020.0  1.0            NaN           NaN             1
            2.0            NaN           NaN             1
            3.0            NaN           NaN             1
...                        ...           ...           ...
110002040.0 11.0           NaN           NaN             1
110002041.0 11.0           NaN           NaN             1
110002042.0 11.0           1.0           0.0             0
110002045.0 11.0           NaN           NaN             1
110002057.0 11.0           NaN           NaN             1

[17874 rows x 3 columns]

In [123]:
if df_h_c["HLS0900b"] == 1:
    df_h_c["HLS0900b_d"] = 1
elif (df_h_c["HLS0900b"]==2 | df_h_c["HLS0900a"]==1):
    df_h_c["HLS0900b_d"] = 0
else:
    df_h_c["HLS0900b_d"] =np.nan

In [ ]:
def create_dummies_depr(dummies_h=None):
    df_h = pd.read_pickle(BLD / "HHENDDAT_clean.pickle")
    if dummies_h is None:
        dummies_h = Path(SRC / "data_management/dummies/HHENDDAT_dummies.yaml")
    with open(dummies_h) as stream:
        dummies_h = yaml.safe_load(stream)["deprivation"]
    for dummies in dummies_h:
        df_h[df_h[f"{dummies}_b"] == 1,"HLS0900_dummies"]=1
        df_h[df_h[f"{dummies}_b"] == 2 ,"HLS0900_dummies"]=0
        df_h[df_h[f"{dummies}_a"] == 1 ,"HLS0900_dummies"]=0
    return df_h


In [168]:
df_h_c.loc[df_h_c["HLS0900b"] == 1,"HLS0900b_d"]=1

In [32]:
 # df = pd.concat([df, pd.get_dummies(df['YEAR'])], axis=1)
 # It is better to include inplace=True when writing the function
sex_dummy = pd.get_dummies(df_p_c['sex'],prefix="sex",).rename(columns = {'sex_1.0': 'sex_male', 'sex_2.0': 'sex_female'})
salary_dummy=pd.get_dummies(df_p_c["salary"],prefix="salary",).rename(columns = {'salary_1.0': 'salary_emp', 'salary_2.0': 'salary_nemp'})
df_p_c['doc_dummy']=(df_p_c['PG0100'] > 0).astype(int)
social_media_dummy=pd.get_dummies(df_p_c['PSM0100'],prefix="soc_med").rename(columns = {'soc_med_1.0': 'soc_med_use', 'soc_med_2.0': 'soc_med_no_use'})
close_friend_dummy= pd.get_dummies(df_p_c['PSK0100'],prefix="close_friend").rename(columns = {'close_friend_1.0': 'close_friend', 'close_friend_2.0': 'close_friend_no'})
# HH variables
alg2_dummy=pd.get_dummies(df_h_c['hh_alg2'],prefix="hh_alg2").rename(columns = {'hh_alg2_1.0': 'hh_alg2_receiving', 'hh_alg2_2.0': 'hh_alg2_notreceiving'})
child_benefit_dummy=pd.get_dummies(df_h_c['child_benefit'],prefix="child_benefit").rename(columns = {'child_benefit_1.0': 'child_benefit_receive', 'child_benefit_2.0': 'child_benefit_notreceiving'})

In [63]:
def create_dummies(dummies_p= None, dummies_h=None):
    df_p = pd.read_pickle(BLD/"PENDDAT_clean.pickle")
    df_h = pd.read_pickle(BLD/"HHENDDAT_clean.pickle")
    if dummies_p == None:
        dummies_p=Path(SRC / "data_management/dummies/PENDDAT_dummies.yaml")
    if dummies_h == None:
        dummies_h=Path(SRC / "data_management/dummies/HHENDDAT_dummies.yaml")
    with open(dummies_p, 'r') as stream:
        dummies_p = yaml.safe_load(stream)
    with open(dummies_h, 'r') as stream:
        dummies_h = yaml.safe_load(stream)
    for dummies in dummies_p:
        if dummies == "PG0100":
            df_p[f"{dummies}_d"] = (df_p[f'{dummies}'] > 0).astype(int)    
        else:
            df_p=pd.concat([df_p,pd.get_dummies(df_p[f'{dummies}'],prefix=f"{dummies}").rename(columns = {f'{dummies}_1.0': f'{dummies}_d'}) ], axis=1).drop(f'{dummies}_2.0', axis=1)
    for dummies in dummies_h:
        df_h=pd.concat([df_h,pd.get_dummies(df_h[f'{dummies}'],prefix=f"{dummies}").rename(columns = {f'{dummies}_1.0': f'{dummies}_d'}) ], axis=1).drop(f'{dummies}_2.0', axis=1)
    return (df_p , df_h)

In [67]:
 pd.get_dummies(df_p_c['sex'],prefix="sex",).rename(columns = {'sex_1.0': 'sex_male', 'sex_2.0': 'sex_female'})

sex_male  sex_female
hh_id       wave                      
10000019.0  1.0          0           1
            1.0          1           0
            3.0          0           1
10000020.0  1.0          1           0
            1.0          0           1
...                    ...         ...
110002041.0 11.0         1           0
110002042.0 11.0         1           0
            11.0         1           0
110002045.0 11.0         1           0
110002057.0 11.0         1           0

[25474 rows x 2 columns]

In [64]:
df_p_dummies,df_h_dummies=create_dummies()

In [66]:
df_p_dummies

p_id  survey_year  survey_mon  sex   age  \
hh_id       wave                                                     
10000019.0  1.0   1.000002e+09         2007           5  2.0  36.0   
            1.0   1.000002e+09         2007           5  1.0  39.0   
            3.0   1.000002e+09         2009           3  2.0  38.0   
10000020.0  1.0   1.000002e+09         2007           4  1.0  66.0   
            1.0   1.000002e+09         2007           4  2.0  61.0   
...                        ...          ...         ...  ...   ...   
110002041.0 11.0  1.100020e+10         2017           5  1.0  29.0   
110002042.0 11.0  1.100020e+10         2017           5  1.0  48.0   
            11.0  1.100020e+10         2017           5  1.0  16.0   
110002045.0 11.0  1.100020e+10         2017           7  1.0  36.0   
110002057.0 11.0  1.100021e+10         2017           8  1.0  32.0   

                  religiosity  PA0100  PA0300  PA0800  PA0900  ...  b5_ext  \
hh_id       wave                                               ...           
10000019.0  1.0           2.0     8.0    10.0     8.0     6.0  ...     NaN   
            1.0           2.0     8.0     7.0     8.0     7.0  ...     NaN   
            3.0           NaN     9.0     7.0     8.0     7.0  ...     NaN   
10000020.0  1.0           NaN     8.0     5.0     4.0     3.0  ...     NaN   
            1.0           3.0     2.0     5.0     NaN     3.0  ...     NaN   
...                       ...     ...     ...     ...     ...  ...     ...   
110002041.0 11.0          2.0    10.0     4.0     8.0     5.0  ...     NaN   
110002042.0 11.0          3.0     1.0     5.0     8.0     2.0  ...     NaN   
            11.0          3.0    10.0     4.0     4.0     7.0  ...     NaN   
110002045.0 11.0          2.0     8.0     7.0     7.0     5.0  ...     NaN   
110002057.0 11.0          3.0     7.0     7.0     5.0     4.0  ...     NaN   

                  b5_agree  b5_consc  b5_neu  b5_open  sex_d  salary_d  \
hh_id       wave                                                         
10000019.0  1.0        NaN       NaN     NaN      NaN      0         0   
            1.0        NaN       NaN     NaN      NaN      1         0   
            3.0        NaN       NaN     NaN      NaN      0         0   
10000020.0  1.0        NaN       NaN     NaN      NaN      1         0   
            1.0        NaN       NaN     NaN      NaN      0         0   
...                    ...       ...     ...      ...    ...       ...   
110002041.0 11.0       NaN       NaN     NaN      NaN      1         0   
110002042.0 11.0       NaN       NaN     NaN      NaN      1         0   
            11.0       NaN       NaN     NaN      NaN      1         0   
110002045.0 11.0       NaN       NaN     NaN      NaN      1         0   
110002057.0 11.0       NaN       NaN     NaN      NaN      1         0   

                  PG0100_d  PSM0100_d  PSK0100_d  
hh_id       wave                                  
10000019.0  1.0          1          0          1  
            1.0          1          0          1  
            3.0          0          0          1  
10000020.0  1.0          0          0          0  
            1.0          1          0          0  
...                    ...        ...        ...  
110002041.0 11.0         1          0          1  
110002042.0 11.0         1          0          1  
            11.0         0          0          0  
110002045.0 11.0         1          0          1  
110002057.0 11.0         1          0          1  

[25474 rows x 102 columns]

In [34]:
import yaml
with open("C:\Project\EPP_project\pass_data_preparation\src\data_management\PENDDAT\PENDDAT_dummies.yaml", 'r') as stream:
    out = yaml.safe_load(stream)
out

['sex', 'salary', 'PG0100', 'PSM0100', 'PSK0100']

In [ ]:
new_names=pd.read_csv("src/data_management/PENDDAT/penddat_renaming.csv",sep=";")["new_name"]

In [ ]:
renaming_dict=dict(zip(df.columns,new_names))

In [ ]:
renaming_dict = {k: v for k, v in renaming_dict.items() if pd.notna(v)}

In [ ]:
def _renaming_columns(path):
    df=pd.read_stata(path,convert_categoricals=False)
    new_names=pd.read_csv("src/data_management/PENDDAT/penddat_renaming.csv",sep=";")["new_name"]
    renaming_dict=dict(zip(df.columns,new_names))
    renaming_dict = {k: v for k, v in renaming_dict.items() if pd.notna(v)}
    return df.rename(columns=renaming_dict)

In [ ]:
def _replacing_negative(df,negatives=list(range(-1,-11,-1))):
    for i in negatives:
        df[df==i]=np.nan
    return df


In [4]:
import os
from pathlib import Path
import glob

In [12]:
files = [file for file in glob.glob("C:\Project\EPP_project\pass_data_preparation\src\original_data\*")]

In [1]:
def get_names_dataset(path="C:\Project\EPP_project\pass_data_preparation\src\original_data"):
    a = r"\*"
    files = list(glob.glob(path + f"{a}"))
    #[file for file in glob.glob(path+f"\*")]
    name=[]
    for i in range(len(files)):
        if any(x.isupper() for x in files[i].split("\\")[-1].split("_")[0]):
            name.append(files[i].split("\\")[-1].split("_")[0])
        else:
            #a=files[i].split("\\")[6].split("_")[0:2]
            name.append(files[i].split("\\")[-1].split("_")[0]+"_"+files[i].split("\\")[-1].split("_")[1])
    return name 
    